Setup API Key

In [8]:
from getpass import getpass
import os
OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Query

In [75]:
query = "What is DeMorgan's Theorem?"

Imports

In [62]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

Embedd File into Documents

In [74]:
loader = PyPDFLoader("../docs/brown_241.pdf")
pages = loader.load_and_split(
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size = 800,
        chunk_overlap = 20,
        length_function = len,
    )
)

embeddings = OpenAIEmbeddings()
chroma_index = Chroma.from_documents(pages, embeddings, persist_directory="../chroma_index")
chroma_index.persist()

Using embedded DuckDB with persistence: data will be stored in: ../chroma_index
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')).


Search & Fetch Documents for Best Match to Query

In [77]:
chroma_index_loaded = Chroma(persist_directory="../chroma_index", embedding_function=embeddings)
doc = chroma_index_loaded.similarity_search(query, k=2)
for d in doc:
    print(d.page_content)

Using embedded DuckDB with persistence: data will be stored in: ../chroma_index


2.5 Boolean Algebra 35
10a.x·y=y·x Commutative
10b.x+y=y+x
11a.x·(y·z)=(x·y)·z Associative
11b.x+(y+z)=(x+y)+z
12a.x·(y+z)=x·y+x·z Distributive
12b.x+y·z=(x+y)·(x+z)
13a.x+x·y=x Absorption
13b.x·(x+y)=x
14a.x·y+x· y=x Combining
14b.(x+y)·(x+ y)=x
15a. x·y= x+y DeMorgan’ s theorem
15b. x+y= x·y
16a.x+ x·y=x+y
16b.x·( x+y)=x·y
17a.x·y+y·z+ x·z=x·y+ x·z Consensus
17b.(x+y)·(y+z)·( x+z)=(x+y)·( x+z)
Again, we can prove the validity of these properties either by perfect induction or by
performing algebraic manipulation. Figure 2.13 illustrates how perfect induction can be
used to prove DeMorgan’s theorem, using the format of a truth table. The evaluation of
left-hand and right-hand sides of the identity in 15agives the same result.
then we should ask whether these gates can be used directly in the synthesis of logic circuits.
In Section 2.5 we introduced DeMorgan’s theorem. Its logic gate interpretation is shown in
Figure 2.26. Identity 15ais interpreted in part (a) of theﬁgure. It speciﬁes

Query LLM given returned document

In [78]:
chain = load_qa_chain(llm=OpenAI(), chain_type="stuff")
chain.run(input_documents=doc, llm=OpenAI(), question=query)

# chain = qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=doc.as_retriever())
# qa.run(query)

" DeMorgan's Theorem states that x·y= x+y and x+y= x·y."

In [79]:
from langchain.llms import LlamaCpp

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

chain = load_qa_chain(llm = LlamaCpp(model_path="../models/ggml-vic13b-q5_1.bin",
                                    callback_manager=callback_manager, verbose=True, n_ctx=2048),
                                 chain_type="stuff")

chain.run(input_documents=doc, question=query)

llama.cpp: loading model from ../models/ggml-vic13b-q5_1.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =  85.08 KB
llama_model_load_internal: mem required  = 11359.04 MB (+ 1608.00 MB per state)
llama_init_from_file: kv self size  = 1600.00 MB
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


 DeMorgan's theorem is an important result in Boolean algebra that states that the negation of a logical OR operation is equivalent to a logical AND operation of the negations, and vice versa. This is expressed mathematically as the identities 15a and 15b in the text: x ⊕ y = (¬x ∧ ¬y) and ¬(x ⊕ y) = (¬x ∧ ¬y). These identities have important implications for the design of digital circuits, as they provide a way to simplify complex logical expressions.

" DeMorgan's theorem is an important result in Boolean algebra that states that the negation of a logical OR operation is equivalent to a logical AND operation of the negations, and vice versa. This is expressed mathematically as the identities 15a and 15b in the text: x ⊕ y = (¬x ∧ ¬y) and ¬(x ⊕ y) = (¬x ∧ ¬y). These identities have important implications for the design of digital circuits, as they provide a way to simplify complex logical expressions."